# **Name : Paras Raut**
# **Email : parasraut2511@gmail.com**
# **Ph No. : 9356375273**
# **Linked : https://www.linkedin.com/in/paras-raut/**
# **Github : https://github.com/parasraut21**

# How To Setup The file

**Step 1: Open Google Colab**
- Open your web browser and go to Google Colab.

**Step 2: Create a New Notebook**
- Click on "File" in the top-left corner, then select "New Notebook" from the dropdown menu.

**Step 3: Copy and Paste Code Blocks**
- Copy each code block provided and paste them into separate cells in your Colab notebook. To create a new cell, you can either press the "+ CODE" button or use the keyboard shortcut Ctrl + M B.

**Step 4: Run the Code Blocks**
- Run each code block by clicking the play button on the left side of the cell or using the keyboard shortcut Shift + Enter. Make sure to run the code blocks in order, starting from the first one.

**Step 5: Upload the Input File**
- Upload the "Input.xlsx" file by clicking on the file icon in the left sidebar, then selecting "Upload" and choosing your input Excel file.

**Step 6: Verify Output**
- After running the final code block, the output Excel file, positive and negative words files, and stopwords files will be downloaded. You can verify these files in your local machine's download folder.

# Import necessary libraries

In [10]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
from datetime import datetime
from google.colab import files

# Download the necessary NLTK resources

In [11]:
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Specify file paths

In [12]:
input_file_path = 'Input.xlsx'
output_file_name = f"Output_Data_Structure_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
positive_words_file = "Positive_Words.txt"
negative_words_file = "Negative_Words.txt"
stopwords_file_prefix = "Stopwords_"


# Load the input.xlsx file

In [14]:

df_input = pd.read_excel(input_file_path)

# Create DataFrames for the output data
columns = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE',
           'POLARITY SCORE', 'SUBJECTIVITY SCORE',
           'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
           'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE']
df_output = pd.DataFrame(columns=columns)

positive_words = set()
negative_words = set()
stopwords_by_type = {}


In [15]:
df_input

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


# Function to extract article text from a URL

In [22]:
def extract_article_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()


        soup = BeautifulSoup(response.text, 'html.parser')

        article_title = soup.find('h1').text
        article_text = ' '.join([p.text for p in soup.find_all('p')])

        return article_title, article_text

    except Exception as e:
        print(f"Error extracting data from {url}: {e}")
        return None, None


# Function to compute text analysis variables

In [23]:
def analyze_text(text):

    sentences = sent_tokenize(text)
    words = word_tokenize(text)


    sia = SentimentIntensityAnalyzer()


    pos_score = sia.polarity_scores(text)['pos']
    neg_score = sia.polarity_scores(text)['neg']
    polarity_score = pos_score - neg_score
    subjectivity_score = sia.polarity_scores(text)['compound']


    avg_sentence_length = sum(len(sent.split()) for sent in sentences) / len(sentences)


    complex_words = [word for word in words if len(word) > 2]  # Filter out short words
    percentage_complex_words = (len(complex_words) / len(words)) * 100


    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)


    avg_words_per_sentence = len(words) / len(sentences)

    # Extract positive and negative words
    positive_words.update([word.lower() for word in words if sia.polarity_scores(word)['compound'] >= 0.5])
    negative_words.update([word.lower() for word in words if sia.polarity_scores(word)['compound'] <= -0.5])



    return pos_score, neg_score, polarity_score, subjectivity_score, avg_sentence_length, \
           percentage_complex_words, fog_index, avg_words_per_sentence


# Iterate through the DataFrame and perform text analysis for each URL

In [24]:
for index, row in df_input.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Extract article text
    title, text = extract_article_text(url)

    if title and text:
        # Perform text analysis
        pos_score, neg_score, polarity_score, subjectivity_score, avg_sentence_length, \
        percentage_complex_words, fog_index, avg_words_per_sentence = analyze_text(text)


        df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,
                                                avg_sentence_length, percentage_complex_words, fog_index,
                                                avg_words_per_sentence], index=columns), ignore_index=True)

        print(f"Text extracted and analyzed for {url_id}.")

    else:
        print(f"Failed to extract text for {url_id}.")


<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


Text extracted and analyzed for blackassign0001.


<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


Text extracted and analyzed for blackassign0002.


<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


...

<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


Text extracted and analyzed for blackassign0098.


<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


Text extracted and analyzed for blackassign0099.
Text extracted and analyzed for blackassign0100.


<ipython-input-24-8f47563a9e0d>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append(pd.Series([url_id, url, pos_score, neg_score, polarity_score, subjectivity_score,


# Save the final DataFrame to the output Excel file

In [25]:
df_output.to_excel(output_file_name, index=False)

# Save positive words to a file
with open(positive_words_file, 'w') as file:
    file.write('\n'.join(sorted(positive_words)))

# Save negative words to a file
with open(negative_words_file, 'w') as file:
    file.write('\n'.join(sorted(negative_words)))

# Save stopwords to separate files based on types

In [20]:
stopwords = nltk.corpus.stopwords.words('english')

for word in stopwords:
    word_type = nltk.pos_tag([word])[0][1]
    stopwords_by_type.setdefault(word_type, []).append(word)

for word_type, stopwords_list in stopwords_by_type.items():
    stopwords_file = f"{stopwords_file_prefix}{word_type}.txt"
    with open(stopwords_file, 'w') as file:
        file.write('\n'.join(sorted(stopwords_list)))


# Trigger download for all files

In [21]:
files.download(output_file_name)
files.download(positive_words_file)
files.download(negative_words_file)

for word_type in stopwords_by_type:
    stopwords_file = f"{stopwords_file_prefix}{word_type}.txt"
    files.download(stopwords_file)

print("Files downloaded successfully.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Files downloaded successfully.
